In [1]:
#!/usr/bin/env python

In [2]:
import os
import sys
if os.path.exists('/home/chieh/code/wPlotLib'):
	sys.path.insert(0,'/home/chieh/code/wPlotLib')
if os.path.exists('/home/chieh/code/wuML'):
	sys.path.insert(0,'/home/chieh/code/wuML')

In [3]:
import wuml
import numpy as np
import torch
import torch.nn as nn

This code shows how you can mix and match different networks<br>
This network simultaneously minimize CE and MSE loss

In [4]:
def status_printing(all_losses, epoch, lr):
	[total_loss, CE_loss, Regress_loss] = all_losses
	txt = '\tepoch: %d, Tloss: %.4f, CEloss: %.4f, MSELoss: %.4f, Learning Rate: %.8f'%((epoch+1), total_loss, CE_loss, Regress_loss, lr)
	wuml.write_to_current_line(txt)

You can also control the behavior of the network on call

In [5]:
def network_behavior_on_call(all_data, all_networks):
	net1 = all_networks[0]
	net2 = all_networks[1]
#
	#	the 1st 2 items of all_data will always be X, and y
	#	the rest will be what you include
	X = all_data[0]
	y = all_data[1]
	y2= all_data[2]
#
	ŷₐ = net1(X)
	ŷᵦ = net2(ŷₐ)
#
	labels = wuml.softmax(ŷₐ, turn_into_label=True)
	return [labels, ŷᵦ]

In [6]:
def costFunction(all_data, all_networks):	
	net1 = all_networks[0]
	net2 = all_networks[1]
#
	#	the 1st 3 items of all_data will always be X, y, index
	#	the rest will be what you include
	X = all_data[0]
	y = all_data[1]
	indx = all_data[2]
	y2= all_data[3]
#
	# run data through the networks
	ŷₐ = net1(X)
	ŷᵦ = net2(ŷₐ)
#
	CE_loss = wuml.CrossEntropyLoss(y, ŷₐ)
	Regress_loss = wuml.MSELoss(y2, ŷᵦ)
	total_loss = 0.1*CE_loss + Regress_loss
#
	return [total_loss, CE_loss, Regress_loss]

In [7]:
def optimizer_steps_order(all_optimizers):
	#	The optimizers are in the order of the network structure you originally defined
	opt1 = all_optimizers[0]
	opt2 = all_optimizers[1]
#
	opt2.step()
	opt1.step()

This data has both regression and classification labels (3 classes)<br>
the network will train on both labels by<br>
	using the 1st network to get 3 softmax outputs, <br>
	from the 1st network, it will connect to the 2nd network, <br>
		expand to width of 5 and compress down to 1 for regression

In [8]:
data = wuml.wData(xpath='../../data/wine.csv', ypath='../../data/wine_label.csv', 
					extra_data='../../data/wine_regress_label.csv', 
					preprocess_data='center and scale', 
					 batch_size=16, label_type='discrete')
Y2 = data.extra_data_dictionary['numpy'][0]

In [9]:
netStructureList = []
netStructureList.append([(100,'relu'),(3,'none')])
netStructureList.append([(50,'relu'),(1,'none')])
netInputDimList = [13, 3]

In [10]:
cNet = wuml.combinedNetwork(data, netStructureList, netInputDimList, costFunction, 
							optimizer_steps_order=optimizer_steps_order,
							max_epoch=1000, on_new_epoch_call_back=status_printing,
							network_behavior_on_call=network_behavior_on_call,
							Y_dataType=torch.LongTensor, extra_dataType=[torch.FloatTensor]) 
cNet.fit()
[labels, ŷᵦ] = cNet(data)

	epoch: 1001, Tloss: 0.0025, CEloss: 0.0240, MSELoss: 0.0001, Learning Rate: 0.00050000



Note that we will save this network for later use, check load_use_network.py file 

In [11]:
wuml.save_torch_network(cNet, './ComplexNet.pk')

In [12]:
CR = wuml.summarize_classification_result(data.Y, labels)
#wuml.jupyter_print('\nAccuracy : %.3f\n\n'%CR.avg_error())

,y,ŷ,Δy
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


In [13]:
SR = wuml.summarize_regression_result(Y2, ŷᵦ)
Reg_result = SR.true_vs_predict(print_out=False)
wuml.jupyter_print(Reg_result, display_all_rows=True)

,y,ŷ,Δy
0,-0.970,-0.982000,0.012
1,-16.928,-16.955999,0.028
2,13.649,13.648000,0.001
3,4.961,4.944000,0.017
4,25.405,25.365000,0.040
...,...,...,...
173,35.462,35.410999,0.051
174,21.143,21.084999,0.058
175,13.592,13.541000,0.051
176,3.000,2.957000,0.043


,y,ŷ,Δy
0,-0.970,-0.982000,0.012
1,-16.928,-16.955999,0.028
2,13.649,13.648000,0.001
3,4.961,4.944000,0.017
4,25.405,25.365000,0.040
5,0.162,0.140000,0.022
6,1.438,1.420000,0.018
7,11.056,11.049000,0.007
8,-12.001,-12.031000,0.030
9,-12.014,-12.047000,0.033
